In [ ]:
################################################################################
# Author 1:      Firstname Lastname
# MatNr 1:       01234567
# Author 2:      Firstname Lastname
# MatNr 2:       01234567
# Author 3:      Jakob Marktl
# MatNr 3:       12335939
# File:          assignment2.ipynb
# Description: ... short description of the file ...
# Comments:    ... comments for the tutors ...
#              ... can be multiline ...
################################################################################

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import pycountry
import pycountry_convert

Alles in einem File, programmieren

Robert:

David:

In [1]:
print("David hat was gemacht")

David hat was gemacht


Jakob:

In [ ]:
FishBoats = pd.read_csv('Fish_Boote.csv')
FishInland = pd.read_csv('Fish_Inland.csv')

FishBoatsSorted = FishBoats.sort_values(by='Reference area', ascending=True)
referenceArea = FishBoatsSorted['Reference area'].tolist()
timePeriod = FishBoatsSorted['TIME_PERIOD'].tolist()
obsValue = FishBoatsSorted['OBS_VALUE'].tolist()


Data = pd.DataFrame({
    'Country': referenceArea,
    'Absolute_Value': obsValue,
    'TimePeriod': timePeriod
})

filtered_Data = Data[Data['TimePeriod'] == 2010]

# Create a scatter plot using Plotly
fig = px.scatter(filtered_Data, x='Country', y='Absolute_Value', 
                 title="Scatter Plot of Amount of Vessels in each Country",
                 labels={'Time_Period': 'Time Period (Years)', 'Absolute_Value': 'Absolute Value (Log)'})
fig.update_yaxes(type='log')
# Show the plot
fig.show()